# Guide to Google Analytics with Python

By Woratana Ngarmtrakulchol [ http://www.byperth.com ]

## Getting started

1. Install Google API Client

In [1]:
# Install Google API Client
#https://medium.com/dataly-data-science-thailand/google-analytics-api-how-to-get-data-from-google-analytics-with-python-in-jupyter-notebook-with-85483dd73e22
!pip install --upgrade google-api-python-client

  Running setup.py bdist_wheel for httplib2: started
  Running setup.py bdist_wheel for httplib2: finished with status 'done'
  Stored in directory: C:\Users\mathu_gopalan\AppData\Local\pip\Cache\wheels\1b\9c\9e\1f6fdb21dbb1fe6a99101d697f12cb8c1fa96c1587df69adba
Successfully built httplib2


Then follow the tutorial below:
2. Create new service account
3. Download key file (got after creating service) and place in the same folder as this file
4. Add service email to SERVICE_ACCOUNT_EMAIL below
5. Add service email as user to the Google Analytics

Tutorial: https://developers.google.com/analytics/devguides/reporting/core/v4/quickstart/service-py

In [3]:
!pip install --upgrade oauth2client

In [3]:
"""Hello Analytics Reporting API V4."""

import argparse
import pandas as pd
import numpy as np
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
KEY_FILE_LOCATION = '.\My Project 26766-ae554bd965b7.p12'
SERVICE_ACCOUNT_EMAIL = 'gatest@valid-progress-217020.iam.gserviceaccount.com'
VIEW_ID = '172386207'

The sample tutorial below is from the tutorial

Basically, we get the data from Google Analytics from get_report() function below. We can adapt the input in `reportRequests` to fit our query.

## Reference for building query
V4 Query Reference: https://developers.google.com/analytics/devguides/reporting/core/v4/basics

And we can get some ideas from V3 API Common Query (Old Version): https://developers.google.com/analytics/devguides/reporting/core/v3/common-queries

But we will need to adapt the query a little bit to fit V4 API.

In [9]:
def initialize_analyticsreporting():
  """Initializes an analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """

  credentials = ServiceAccountCredentials.from_p12_keyfile(
    SERVICE_ACCOUNT_EMAIL, KEY_FILE_LOCATION, scopes=SCOPES)

  http = credentials.authorize(httplib2.Http())

  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics


def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
  return analytics.reports().batchGet(
        # Get sessions number from the last 7 days
#       body={
#         'reportRequests': [
#         {
#           'viewId': VIEW_ID,
#           'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
#           'metrics': [{'expression': 'ga:sessions'}]
#         }]
#       }
      # Get posts from last 7 days
      body={
          'reportRequests': [
              {
                  'viewId': VIEW_ID,
                  'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
                  'metrics': [
                      {'expression': 'ga:pageviews'},
                      {'expression': 'ga:uniquePageviews'},
                      {'expression': 'ga:timeOnPage'},
                      {'expression': 'ga:bounces'},
                      {'expression': 'ga:entrances'},
                      {'expression': 'ga:exits'}
                  ],
                  "dimensions": [
                      {"name": "ga:pagePath"}
                  ],
                  "orderBys": [
                      {"fieldName": "ga:pageviews", "sortOrder": "DESCENDING"}
                  ]
              }
          ]
      }
  ).execute()


def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response"""

  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])

    for row in rows:
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])

      for header, dimension in zip(dimensionHeaders, dimensions):
        print ( header + ': ' + dimension )

      for i, values in enumerate(dateRangeValues):
        print ('Date range (' + str(i) + ')' )
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print ( metricHeader.get('name') + ': ' + value )


def main():
    analytics = initialize_analyticsreporting()
    response = get_report(analytics)
    #df = pd.DataFrame(response)
    #print(df.head())
    print_response(response)

if __name__ == '__main__':
  main()


ga:pagePath: /en-gb/oral-health/
Date range (0)
ga:pageviews: 1213
ga:uniquePageviews: 626
ga:timeOnPage: 40210.0
ga:bounces: 113
ga:entrances: 520
ga:exits: 269
ga:pagePath: /en-gb/samples/
Date range (0)
ga:pageviews: 1137
ga:uniquePageviews: 518
ga:timeOnPage: 44709.0
ga:bounces: 23
ga:entrances: 153
ga:exits: 205
ga:pagePath: /en-gb/registration/
Date range (0)
ga:pageviews: 497
ga:uniquePageviews: 381
ga:timeOnPage: 73978.0
ga:bounces: 31
ga:entrances: 84
ga:exits: 101
ga:pagePath: /en-gb/vpv/register/complete
Date range (0)
ga:pageviews: 252
ga:uniquePageviews: 237
ga:timeOnPage: 33958.0
ga:bounces: 0
ga:entrances: 2
ga:exits: 88
ga:pagePath: /samples/
Date range (0)
ga:pageviews: 200
ga:uniquePageviews: 82
ga:timeOnPage: 5578.0
ga:bounces: 2
ga:entrances: 33
ga:exits: 28
ga:pagePath: /en-gb/my-profile/personal-detail/
Date range (0)
ga:pageviews: 184
ga:uniquePageviews: 98
ga:timeOnPage: 2347.0
ga:bounces: 1
ga:entrances: 2
ga:exits: 9
ga:pagePath: /en-gb/thank-you/?errorcode=0
